Frame: functions for start or end of the game. prints 40 asterics (*) for framing the game.

Basic concepts:
1. Input: Name, Level Beginner 1-10,Medium 1-100, Expert 1-1000
2. Set: 20 questions in the test
3. Starting the trainning:

Questions (list):
A question (class Question). 
1. question number - key
2. operation (+,-,*)
3. first number
4. second number
5. correct answer
6. user's answer.


Questions orchestrator main object according to object oriented methodology with a list of questions. The orchestrator manages the whole test including student's name, test level, start/end time, questions generation and answers
  a. A Function that picks two random numbers from 1-10, 1-100 or 1-1000 according the level, and generate exercises. 
  The basic game contains a single arithmetic operation (+,-,*).

  An enhanced version of the game may contain two or more arithmetic operations, regular expression for email, history per (user,level), add audio for correct and wrong answer consequently.
  
  The exercises will run in a loop until 20 .(Demo: until 5)

  Operation is chosen randomly '+','-','*'

  For applying this limitation each iteration of the loop checks counter of operation exercises limit.

  b. Another function that counts the score for every correct answer and retrieve the total score of the trainee.

  Export results to a .txt file via gmail including:
  Start DateTime, End DateTime, User's name, Level, List of questions
  alongside List of correct answers and a List of user's answers, as well as the final test result.

  Timer object schedules the test and exists when time is up even if test is not finished yet.

  Auxiliary Python Modules:
  1. Datetime - datetime, delta, 
  2. Random - generating numbers,
  3. IO - input output - both interactive print to console and print to file stream
  4. Classes - each instance of question is implemented as a class object
  5. RE - check validation of name as alpha string of 30 chars
  6. Audio - correct answer and wrong answer will have appropriate sound and notification.

In [ ]:
from datetime import datetime,date,timedelta,time
from dataclasses import dataclass, field
from collections import namedtuple  
from random import randint
import random
import re
import io
import sys
#import simpleaudio as sa
#import winsound


class Point(tuple):
    def __init__(self, x, y):
        return tuple.__init__(Point, (x, y))

 ##########################################################
class Timer:
  duration: int # minutes
  user_time: int
  start_dt: datetime
  start_time: time
  end_time: time
  utc_offset: int
  start_t_str: str
  end_t_str: str

  def __init__(self,utc_offset=0):
    self.duration = 1 #minutes
    self.user_time = 0
    self.start_dt = datetime.now()
    self.start_time = self.start_dt.time()
    self.end_time = datetime.now().time() 
    self.utc_offset = utc_offset
    self.start_t_str = ''
    self.end_t_str = ''

  def set_timer(self):
  
    self.start_dt = datetime.utcnow()  + timedelta(hours=self.utc_offset) 
    self.end_time = (self.start_dt + timedelta(minutes=self.duration)).time()

    self.start_t_str = time.strftime(self.start_dt.time(), '%H:%M')
    self.end_t_str = time.strftime(self.end_time, '%H:%M')
    
    print(f"Start Time: {self.start_t_str} End Time: {self.end_t_str}")


  def is_time_left(self):

    current_time = datetime.utcnow() + timedelta(hours=self.utc_offset)
    current_t_str = time.strftime(current_time.time(), '%H:%M')
    return current_time.time() <= self.end_time

  def finish_time(self):

    current_time = datetime.utcnow() + timedelta(hours=self.utc_offset)
    current_t_str = time.strftime(current_time.time(), '%H:%M')
    print(f"Finish Time: {current_t_str}")
 ##########################################################  
@dataclass 
class Question:
 
  point: namedtuple('Point', ('x', 'y'))
  level_bounds: int
  operator: str
  answer_text: str
  user_answer: str
  correct_answer_value: int

  def __init__ (self,level_bounds=10):

      self.level_bounds = 0
      self.level_bounds = level_bounds
      self.answer_text = ''
      self.correct_answer_value = 0
      self.user_answer = ''

      Point = namedtuple('Point', ('x', 'y'))
      self.point = Point(0,0)
      self.operator ='+'

  def set_point(self):

      num1 = 0
      num2 = 0
      num1 = randint(1,self.level_bounds)
      num2 = randint(1,self.level_bounds)

      Point = namedtuple('Point', ('x', 'y'))
      self.point = Point(num1, num2)
      self.operator = '!'
      sym =  ['+', '-', '*']
      self.operator = random.choice(sym)
     
    
  def __calc__ (self):
      
      self.get_user_answer()
      #if self.check_correct_answer()=='True'
         # print("")
      

  def get_user_answer(self,level):

    self.user_answer = input('Your answer is...')

    if (level == 'B' or level == 'M'):
        while (self.user_answer.isnumeric() == False):
            self.user_answer = input('Not a valid answer, please enter a positive number...')
    elif (level == 'E'):
        while(True):
            if (self.user_answer[0]=='-' and self.user_answer[::-1][:-1].isnumeric()):
                break
            if self.user_answer.isnumeric() == True:
                break
            self.user_answer = input('Not a valid answer, please enter a positive/negative number...')
        
   
  def calculate_answer(self):
      self.answer_text = f'{self.point[0]}{self.operator}{self.point[1]}'
      self.correct_answer_value = eval(self.answer_text)

        
  def answer_compare(self,x): 
      return [('Correct!') if x==True else ('Wrong!..Good Luck next question')]          
                                      

  def check_correct_answer(self):
    return self.correct_answer_value == int(self.user_answer)
   
    
  def validate_level_bounds(self,level):    
        if (level == 'B' or level == 'M'):           
              if self.correct_answer_value < 0:  
                  return False
              else:                
                  return True
        elif level == 'E':             
              return True
        else:
              return False

  def print_question(self, question_num = 1,result='?'):
      print(f"Question {question_num}:  {self.point[0]} {self.operator} {self.point[1]} = {result}")


 ########################################################## 
@dataclass
class Questionnaire:
  timer: Timer
  name: str
  level: str
  level_bounds: int
  level_dict: dict
  #level_list: list
  total_questions: int
  correct_answers: int 
  questions_list: list
  grade: int

  def __init__ (self, name='user' ,level='B',level_bounds=10): 
      self.name = name
      self.level = level
      self.level_bounds = level_bounds

      self.level_dict =  {'B':("Beginner",10), 
                          'M':("Medium",100), 
                          'E':("Expert",1000)} 

      self.total_questions = 5
      self.correct_answers = 0
      self.questions_list = [] 
      self.grade = 0  
      self.timer = Timer(utc_offset=2) # Israel time is UTC + 2             
      

  def print_header (self, greeting_text): 
      print(40 * "*")
      if (greeting_text is None or greeting_text==''):
          return None

      print(f"* {greeting_text:^36} *") 
      print(40 * "*")
      return None

  def get_details(self):
      self.name = input("Please enter your name: " )
      name_re = re.search(r'[a-zA-Z]{2,10}',self.name)  

      while (name_re is None):
        self.name = input("Please enter valid name of (2-10) letters")
        name_re = re.search(r'[a-zA-Z]{2,10}',self.name)     
      print()
      pass

  def get_level(self):

      nl = '\n'
      star = 15 * '*'
      level_options = ''
      print(f"{'Level':<24} {'Numbers':<20} ")
      print(f"{star:<20} {star:<20}" )
          
      for key, (value1,value2) in self.level_dict.items():
          print(f"{'('+value1[0] +') ':<4}{value1:<20} {'1-' +str(value2):<20}")
          level_options += value1[0]

      level_options_n = ' / '.join(level_options)
      self.level = 'X'
 
      print(f"{nl}")

      while self.level.upper() not in level_options: 
          self.level = input(f'{self.name}, please enter your level... ({level_options_n}): ')       
          valid_level = len([ch for ch in self.level if ch in level_options])
         
          if valid_level == 1:# equivalent to :if self.level.upper() in level_options:     
              self.level_bounds = self.level_dict.get(self.level.upper())[1]
              break
          else:
              print("This is not a legitimate level...")

  def start_test(self): 
        self.timer.set_timer()
        print(f"Test Duration: {str(self.timer.duration)}min  Total Questions: {str(self.total_questions)}")
        self.print_header('')

  def finish_test(self):
        self.timer.finish_time()
        print(f'Grade = {str(self.grade):<4} , Correct Answers: {self.correct_answers} out of {self.total_questions}')
       
  def repeat_questions(self):
        for x in range(1, self.total_questions+1):  # for 5 questions  #range(1, 21): # for 20 questions 
            Question1 = Question(level_bounds=Questionnaire1.level_bounds)
            valid_bounds = False

            while valid_bounds  == False:  #reset negative operands
                #print("Negative!!")
                Question1.set_point() 
                Question1.calculate_answer()
                valid_bounds = Question1.validate_level_bounds(Questionnaire1.level)
    
            Question1.print_question(result='?')
            Question1.get_user_answer(Questionnaire1.level)

            #print("Is Time Left ? " +str(Timer1.is_time_left()))

            self.grade += int(100/self.total_questions) * Question1.check_correct_answer()
            self.questions_list.append(Question1)

  def print_test(self):
   
      answer_mark = lambda x,y: '[V]' if x==y else '[X]'
      self.print_header("Test Review")
  
      for counter, question in enumerate(self.questions_list, 1):
         # answer_mark = lambda x,y: '[V]' if x==y else '[X]'
          correct_answer_value_str = str(question.correct_answer_value)
          print ("{}  Question{:4}: {:16} = {:16} , Your Answer =  {:22}".
                 format( answer_mark(correct_answer_value_str,question.user_answer),
                         counter,question.answer_text,correct_answer_value_str,question.user_answer)) 
                        
          
  def export_test_to_file(self):

      nl = '\n'
      stars = 40 * '*'
      text = '*** Test Review ***'
      level,bound = self.level_dict[self.level]
    
      data = ''
      data  +=  f"{stars}{nl}"
      data  +=  f"user = {self.name} , level = {level} , Grade = {self.grade} {nl}" 
      data  +=  f"start time = {self.timer.start_t_str} , end_time = {self.timer.end_t_str} {nl}"
      data  +=  f"Correct Answers = {self.correct_answers} out of {self.total_questions} {nl}"
      data  +=  f"{stars}{nl}{nl}"    
            
      for counter, question in enumerate(self.questions_list, 1):
          answer_mark = lambda x,y: '[V]' if x==y else '[X]'
          correct_answer_value_str = str(question.correct_answer_value)
          answer_mark_str = answer_mark(correct_answer_value_str,question.user_answer)

          data += f"{answer_mark_str} " \
                  f"Question {counter} : " \
                  f"{question.answer_text} = " \
                  f"{correct_answer_value_str}  , Your Answer = " \
                  f"{question.user_answer} {nl}" 
                        
      # Create the new file
      with open("test_data.txt", 'w') as f:
          try:
              f.write(data)
          except Exception as e:
              print(e)

      f.close()

      import email, smtplib, ssl

      from email import encoders
      from email.mime.base import MIMEBase
      from email.mime.multipart import MIMEMultipart
      from email.mime.text import MIMEText

      subject = "An email with attachment from Python"
      body = "This is an email with attachment sent from Python"
      sender_email = "shel777@gmail.com" 
      receiver_email = "shel777@gmail.com" 
      password = input("Type your password and press enter:")

      # Create a multipart message and set headers
      message = MIMEMultipart()
      message["From"] = sender_email
      message["To"] = receiver_email
      message["Subject"] = subject
      message["Bcc"] = receiver_email  # Recommended for mass emails

      # Add body to email
      message.attach(MIMEText(body, "plain"))

      filename = "test_data.txt" # In same directory as script

      # Open .txt  file in binary mode
      with open(filename, "rb") as attachment:
          # Add file as application/octet-stream
          # Email client can usually download this automatically as attachment
          part = MIMEBase("application", "octet-stream")
          part.set_payload(attachment.read())

      # Encode file in ASCII characters to send by email    
      encoders.encode_base64(part)

      # Add header as key/value pair to attachment part
      part.add_header(
         "Content-Disposition",
          f"attachment; filename= {filename}",
      )

      # Add attachment to message and convert message to string
      message.attach(part)
      text = message.as_string()

      # Log in to server using secure context and send email
      context = ssl.create_default_context()
      with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:   
          try:
              server.login(sender_email, password)
              server.sendmail(sender_email, receiver_email, text)
          except Exception as e:
            print(e)

      print(f"File is sent to mail:{receiver_email}")
          # answer_mark = lambda x,y: '[V]' if x==y else '[X]'

                    
###########################################
# Start Algorithm
###########################################

Questionnaire1 = Questionnaire(name='user',level='B') 
Questionnaire1.print_header('Wellcome to Math Test')
Questionnaire1.get_details()
Questionnaire1.get_level()
Questionnaire1.print_header('Test Started , Good Luck')
Questionnaire1.start_test()
#Questionnaire1.repeat_questions()
current_question = 0

for x in range(1, Questionnaire1.total_questions+1):  # for 5 questions  #range(1, 21): # for 20 questions 
      current_question += 1
      Question1 = Question(level_bounds=Questionnaire1.level_bounds)
      valid_bounds = False

      while valid_bounds  == False:  #reset negative operands
          #print("Negative!!")
          Question1.set_point() 
          Question1.calculate_answer()
          valid_bounds = Question1.validate_level_bounds(Questionnaire1.level)
    
      Question1.print_question(question_num=current_question,result='?')
      Question1.get_user_answer(Questionnaire1.level)
      
      if (Questionnaire1.timer.is_time_left() == False):
          print("Time is up!")
          break
 
      is_answer_correct = Question1.check_correct_answer()

      #if (is_answer_correct):
         # winsound.Beep(1000, 100) 

      Questionnaire1.grade += int(100/Questionnaire1.total_questions) * Question1.check_correct_answer()
      Questionnaire1.correct_answers += int(is_answer_correct) #1

      print(f"{str(Question1.answer_compare(is_answer_correct) )}\n")
      Questionnaire1.questions_list.append(Question1)

Questionnaire1.finish_test()
Questionnaire1.print_test()
Questionnaire1.print_header("Goodbye !")
Questionnaire1.export_test_to_file()


****************************************
*        Wellcome to Math Test         *
****************************************
Please enter your name: Mallaachy

Level                    Numbers              
***************      ***************     
(B) Beginner             1-10                
(M) Medium               1-100               
(E) Expert               1-1000              


Mallaachy, please enter your level... (B / M / E): E
****************************************
*       Test Started , Good Luck       *
****************************************
Start Time: 21:40 End Time: 21:41
Test Duration: 1min  Total Questions: 5
****************************************
Question 1:  72 * 278 = ?


In [ ]:
# RE for validating an Email 

def check(email): 
    eregex = r'^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}+[\.]?\w{2,3}$'
    # pass the regular expression 
    # and the string in search() method 
    if re.search(eregex, email):  
        print("Valid Email")   
    else:  
        print("Invalid Email")  

In [ ]:
#testing
email = 'elad.eliav@gmail.com'
check(email)

error: ignored

In [ ]:
#This function is not callable. 
#def correctsound():
from IPython.display import Audio
tada = Audio('Tada-sound.mp3.mp3', autoplay=True)
tada

In [ ]:

#correct function
def correct():
  print("You Are Correct!")

# Testing for sound function
import string
from IPython.display import Audio
tada = Audio('Tada-sound.mp3.mp3', autoplay=True)
letters = string.ascii_lowercase
x = input('Enter letter')
if x in letters:
  correct()
  tada()

In [ ]:
#Selecting level
#Options = ['A','B','C']
#level = input(f'Select your level{Options}')
#level = level.upper()
#while level != Options:
 # if level in 'ABCabc':
  #  print(f'YOUR LEVEL IS  {level}')
   # break
  #else:
  #  level = input(f'Select your level again please!{Options}')
#print(level)

In [ ]:
# %%writefile test.txt
# resaults = open('test.txt','a+')
#score = 0
#q1 = ''
#def Trainer():
 # from random import randint
 # import random
# Generating two random numbers - level B
 # list1 = []
 # for _ in range(2):
  #  numbers = randint(1,10)
  #  list1.append(numbers)
   # list1 = sorted(list1)

  #random operator

  #sym = ['+', '-', '*']
  #operator = random.choice(sym)

  #print(question1.point)
  #global q1
  #q1 = f'{list1[1]}{operator}{list1[0]}'
  #print(q1)

  #Question and Answer 
  #print(q1)
  #global score
  #print(eval(q1))
  #user_answer = input('Your answer is...')
  # resaults.write(f'{q1}\n')
  #if eval(q1) == int(user_answer):
    #print('Correct!!!\n')
   # score +=5
  #else:
    #print('Good luck next question\n')

In [ ]:
#open('test.txt')
#counter = 1
#for i in range(1,21):
#  print (f'Question {counter}')
 # Trainer()
 # counter +=1
 # print(f'You have got {score} points so far!\n')
#score = 0